In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time
import csv
from numpy import array
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import completeness_score, homogeneity_score, v_measure_score
import imblearn
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.


# <font color=black>Random Over Sampling</font>

In [2]:
df = pd.read_csv('dataset_lof.csv')
dataset = df.copy()
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,0.1
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1.0
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0


In [3]:
col_names = ["Destination_Port",
             "Flow_Duration", 
             "Total_Fwd_Packets",
             "Total_Backward_Packets",
             "Total_Length_of_Fwd_Packets", 
             "Total_Length_of_Bwd_Packets", 
             "Fwd_Packet_Length_Max", 
             "Fwd_Packet_Length_Min", 
             "Fwd_Packet_Length_Mean", 
             "Fwd_Packet_Length_Std",
             "Bwd_Packet_Length_Max", 
             "Bwd_Packet_Length_Min", 
             "Bwd_Packet_Length_Mean", 
             "Bwd_Packet_Length_Std",
             "Flow_Bytes_s", 
             "Flow_Packets_s", 
             "Flow_IAT_Mean", 
             "Flow_IAT_Std", 
             "Flow_IAT_Max", 
             "Flow_IAT_Min",
             "Fwd_IAT_Total", 
             "Fwd_IAT_Mean", 
             "Fwd_IAT_Std", 
             "Fwd_IAT_Max", 
             "Fwd_IAT_Min",
             "Bwd_IAT_Total", 
             "Bwd_IAT_Mean", 
             "Bwd_IAT_Std", 
             "Bwd_IAT_Max", 
             "Bwd_IAT_Min",
             "Fwd_PSH_Flags", 
             "Bwd_PSH_Flags", 
             "Fwd_URG_Flags", 
             "Bwd_URG_Flags", 
             "Fwd_Header_Length", 
             "Bwd_Header_Length",
             "Fwd_Packets_s", 
             "Bwd_Packets_s", 
             "Min_Packet_Length", 
             "Max_Packet_Length", 
             "Packet_Length_Mean", 
             "Packet_Length_Std", 
             "Packet_Length_Variance",
             "FIN_Flag_Count", 
             "SYN_Flag_Count", 
             "RST_Flag_Count", 
             "PSH_Flag_Count", 
             "ACK_Flag_Count", 
             "URG_Flag_Count", 
             "CWE_Flag_Count", 
             "ECE_Flag_Count", 
             "Down_Up_Ratio", 
             "Average_Packet_Size", 
             "Avg_Fwd_Segment_Size", 
             "Avg_Bwd_Segment_Size",
             "Fwd_Avg_Bytes_Bulk", 
             "Fwd_Avg_Packets_Bulk", 
             "Fwd_Avg_Bulk_Rate", 
             "Bwd_Avg_Bytes_Bulk", 
             "Bwd_Avg_Packets_Bulk",
             "Bwd_Avg_Bulk_Rate",
             "Subflow_Fwd_Packets", 
             "Subflow_Fwd_Bytes", 
             "Subflow_Bwd_Packets", 
             "Subflow_Bwd_Bytes",
             "Init_Win_bytes_forward", 
             "Init_Win_bytes_backward", 
             "act_data_pkt_fwd", 
             "min_seg_size_forward",
             "Active_Mean", 
             "Active_Std", 
             "Active_Max", 
             "Active_Min",
             "Idle_Mean", 
             "Idle_Std", 
             "Idle_Max", 
             "Idle_Min",
             "Label_Category_Num"
            ]

In [5]:
dataset.columns = col_names
dataset.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_Category_Num
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1.0
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0.0


In [6]:
dataset["Label_Category_Num"] = dataset.Label_Category_Num.astype(int)
dataset.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label_Category_Num
0,0.000809,0.001476,0.000009,0.000007,0.000005,1.556176e-07,0.001410,0.015054,0.005891,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
1,0.000809,0.000198,0.000005,0.000003,0.000004,1.266300e-07,0.002055,0.021935,0.008585,0.000000,...,5.960467e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
2,0.001221,0.708609,0.000032,0.000021,0.000029,1.769005e-05,0.014746,0.000000,0.008945,0.019363,...,3.725292e-08,0.000009,0.0,0.000009,0.000009,0.7075,0.0,0.7075,0.7075,1
3,0.000809,0.000002,0.000009,0.000007,0.000005,3.264917e-07,0.001209,0.012903,0.005050,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0
4,0.000809,0.001209,0.000005,0.000003,0.000003,1.540919e-07,0.001692,0.018065,0.007070,0.000000,...,3.725292e-08,0.000000,0.0,0.000000,0.000000,0.0000,0.0,0.0000,0.0000,0


In [7]:
benign_df      = dataset[dataset['Label_Category_Num'] == 0]
dos_df         = dataset[dataset['Label_Category_Num'] == 1]
probe_df       = dataset[dataset['Label_Category_Num'] == 2]
ddos_df        = dataset[dataset['Label_Category_Num'] == 3] 
brute_force_df = dataset[dataset['Label_Category_Num'] == 4]
botnet_df      = dataset[dataset['Label_Category_Num'] == 5]
web_attack_df  = dataset[dataset['Label_Category_Num'] == 6]

In [8]:
benign_dos     = pd.concat([benign_df, dos_df])
benign_probe   = pd.concat([benign_df, probe_df])
benign_ddos    = pd.concat([benign_df, ddos_df])
benign_bforce  = pd.concat([benign_df, brute_force_df])
benign_wattack = pd.concat([benign_df, web_attack_df])
benign_botnet  = pd.concat([benign_df, botnet_df])

In [9]:
benign_dos_input      = benign_dos.iloc[:,:-1]
benign_dos_output     = benign_dos.iloc[:,[-1]]

benign_probe_input    = benign_probe.iloc[:,:-1]
benign_probe_output   = benign_probe.iloc[:,[-1]]

benign_ddos_input     = benign_ddos.iloc[:,:-1]
benign_ddos_output    = benign_ddos.iloc[:,[-1]]

benign_bforce_input   = benign_bforce.iloc[:,:-1]
benign_bforce_output  = benign_bforce.iloc[:,[-1]]

benign_wattack_input  = benign_wattack.iloc[:,:-1]
benign_wattack_output = benign_wattack.iloc[:,[-1]]

benign_botnet_input   = benign_botnet.iloc[:,:-1]
benign_botnet_output  = benign_botnet.iloc[:,[-1]]

In [10]:
oversample           = RandomOverSampler(sampling_strategy=0.45)
x_dos, y_dos         = oversample.fit_resample(benign_dos_input, benign_dos_output)

oversample1          = RandomOverSampler(sampling_strategy=0.28)
x_probe, y_probe     = oversample1.fit_resample(benign_probe_input, benign_probe_output)

oversample2          = RandomOverSampler(sampling_strategy=0.23)
x_ddos, y_ddos       = oversample2.fit_resample(benign_ddos_input, benign_ddos_output)

oversample3          = RandomOverSampler(sampling_strategy=0.025)
x_bforce, y_bforce   = oversample3.fit_resample(benign_bforce_input, benign_bforce_output)

oversample4          = RandomOverSampler(sampling_strategy=0.004)
x_wattack, y_wattack = oversample4.fit_resample(benign_wattack_input, benign_wattack_output)

oversample5          = RandomOverSampler(sampling_strategy=0.003)
x_botnet, y_botnet   = oversample5.fit_resample(benign_botnet_input, benign_botnet_output)

C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\deneysel\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_

In [11]:
x_dos     = pd.DataFrame(x_dos)
y_dos     = pd.DataFrame(y_dos)
x_probe   = pd.DataFrame(x_probe)
y_probe   = pd.DataFrame(y_probe)
x_ddos    = pd.DataFrame(x_ddos)
y_ddos    = pd.DataFrame(y_ddos)
x_bforce  = pd.DataFrame(x_bforce)
y_bforce  = pd.DataFrame(y_bforce)
x_wattack = pd.DataFrame(x_wattack)
y_wattack = pd.DataFrame(y_wattack)
x_botnet  = pd.DataFrame(x_botnet)
y_botnet  = pd.DataFrame(y_botnet)

In [12]:
x = pd.concat([x_dos, x_probe, x_ddos, x_bforce, x_wattack, x_botnet])
y = pd.concat([y_dos, y_probe, y_ddos, y_bforce, y_wattack, y_botnet])

In [13]:
data = pd.concat([x, y], axis = 1)

In [14]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,0
0,0.000809,1.475758e-03,0.000009,0.000007,5.426357e-06,1.556176e-07,0.001410,0.015054,0.005891,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.000809,1.980333e-04,0.000005,0.000003,3.953488e-06,1.266300e-07,0.002055,0.021935,0.008585,0.0,...,5.960467e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.000809,1.633333e-06,0.000009,0.000007,4.651163e-06,3.264917e-07,0.001209,0.012903,0.005050,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000809,1.208725e-03,0.000005,0.000003,3.255814e-06,1.540919e-07,0.001692,0.018065,0.007070,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.861494,2.500000e-08,0.000009,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.001010,0.0,...,3.725292e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
data.columns = col_names

In [16]:
dos     = data[data['Label_Category_Num'] == 1]
probe   = data[data['Label_Category_Num'] == 2]
ddos    = data[data['Label_Category_Num'] == 3]
bforce  = data[data['Label_Category_Num'] == 4]
botnet  = data[data['Label_Category_Num'] == 5]
wattack = data[data['Label_Category_Num'] == 6]

In [17]:
data2 = pd.concat([dos, probe, ddos, bforce, wattack, botnet])
len(data2)

2000047

In [18]:
ros = pd.concat([benign_df, data2])
len(ros)

4016226

In [19]:
ros.to_csv('dataset_ros.csv', index=False)